# Library

In [1]:
import re
import torch
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Prep

- 원화, 원/달러 키워드가 들어간 뉴스만 필터링
- 정규표현식 패턴 제거
- 내용 없이 제목만 있는 뉴스 제거
- 하나은행(서울 중구 을지로 하나은행 본점 딜링룸 현황판)/ 북한 원화 뉴스 제거

In [2]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020.csv"
df = pd.read_csv(data_path)

In [3]:
# 중요 키워드 리스트 정의
keywords = ["원화", "원/달러"]

# "달러", "환율", "외환", "외환시장", "통화", "금리", "환전", "외화", "환변동"

# 중요 단어 포함 여부 
def contains_keyword(text):
    if pd.isna(text): 
        return False
    return any(keyword in text for keyword in keywords)

# 필터링 적용 
df = df[df["title"].apply(contains_keyword) | df["content"].apply(contains_keyword)]

# 정규표현식 패턴 정의
patterns_content = [
    r"\[.*?\]",  # 대괄호 안의 내용 및 대괄호 자체 제거
    r"\(.*?\)",  # 소괄호 안의 내용 및 소괄호 자체 제거
    r"[^ ]+@[^ ]+\.[a-z]{2,3}",  # 이메일 주소 제거
    r"▶.*?$",  # ▶로 시작하는 광고성 문구 제거
    r".*?=",  # "=" 이전 모든 내용 삭제 및 "=" 삭제
]

patterns_title = [
    r"\[.*?\]",  # 대괄호 안의 내용 및 대괄호 자체 제거
    r"\(.*?\)",  # 소괄호 안의 내용 및 소괄호 자체 제거
]

# 데이터 정제
for pattern in patterns_content:
    df["content"] = df["content"].apply(lambda x: re.sub(pattern, "", x).strip())

for pattern in patterns_title:
    df["title"] = df["title"].apply(lambda x: re.sub(pattern, "", x).strip())
    
# 중복제거 
df = df.drop_duplicates(subset=["title", "content"], keep="first")

# 마지막 문장을 추출
def remove_last_sentence_if_no_period(text):
    if pd.isna(text):  
        return text
    
    # 문장을 문장부호 기준으로 분리 (". ", "! ", "? ")
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    
    # 문장이 없는 경우 그대로 반환
    if not sentences:
        return text

    # 마지막 문장이 "."으로 끝나지 않으면 삭제
    if not sentences[-1].endswith("."):
        sentences.pop()  # 마지막 문장 제거

    return " ".join(sentences)  # 다시 문장 합치기

# content에만 적용
df["content"] = df["content"].apply(remove_last_sentence_if_no_period)

print(f"{len(df)}개의 뉴스가 남았습니다.")

1155개의 뉴스가 남았습니다.


In [4]:
# content 결측치면 title로 대체
#df["content"] = df.apply(lambda row: row["title"] if pd.isna(row["content"]) or row["content"].strip() == "" else row["content"], axis=1)

# content가 결측값이거나 빈 문자열인 행 제거
df = df[~(df["content"].isna() | (df["content"].str.strip() == ""))]

In [5]:
df = df.dropna(subset=['content'])
print(f"{len(df)}개의 뉴스가 남았습니다.")

611개의 뉴스가 남았습니다.


In [6]:
# 하나은행, 북한 필터링
df = df[~(df["content"].str.contains("하나은행|북한", na=False))]
print(f"{len(df)}개의 뉴스가 남았습니다.")

455개의 뉴스가 남았습니다.


In [7]:
# 한 번 더 필터링
keywords = ["원화", "원/달러"]

def contains_currency_keywords(text):
    if not isinstance(text, str):
        return False
    return any(kw in text for kw in keywords)

df = df[df["title"].apply( contains_currency_keywords) | df["content"].apply( contains_currency_keywords)]
print(f"{len(df)}개의 뉴스가 남았습니다.")

383개의 뉴스가 남았습니다.


In [8]:
df

,date,title,url,content
2,20200101,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치",https://n.news.naver.com/mnews/article/001/001...,외환당국이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...
6,20200101,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율은 롤러코스터를 타는 듯했다. 연중 고점과 저점 차이가 110원 ...
8,20200101,"위험선호 분위기 속 원/달러 환율 1,150원대로 하락 출발",https://n.news.naver.com/mnews/article/001/001...,30일 오전 원/달러 환율이 하락한 채로 출발했다.이날 서울 외환시장에서 원/달러 ...
16,20200102,새해 첫 거래일 원/달러 환율 상승 마감,https://n.news.naver.com/mnews/article/001/001...,새해 첫 거래일인 2일 원/달러 환율이 올랐다.이날 서울 외환시장에서 원/달러 환율...
30,20200103,중동 긴장 고조로 금융시장 불안…주가·환율·금리 '출렁',https://n.news.naver.com/mnews/article/001/001...,미국과 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/달러 ...
...,...,...,...,...
6632,20201204,미국 부양책 기대에 약달러 심화…원/달러 환율 하락 출발,https://n.news.naver.com/mnews/article/001/001...,4일 원/달러 환율이 하락 출발했다.이날 서울 외환시장에서 원/달러 환율은 오전 9...
6634,20201204,10월 경상 흑자 117억弗 달해 원화 강세 지속되나,https://n.news.naver.com/mnews/article/011/000...,경상흑자 올 해 전망치 540억弗 이미 넘겨 호조세 지난 10월 경상수지 흑자가 1...
6716,20201209,"위안화 강세에 원/달러 환율 하루 만에 반락…1,084.8원 마감",https://n.news.naver.com/mnews/article/001/001...,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.이날 서울 외환시장에서 원/달러...
6896,20201218,"원·달러 환율, 장중 1100원 터치…코로나19 악화에 급등 마감",https://n.news.naver.com/mnews/article/015/000...,게티이미지뱅크원·달러 환율이 급등 마감했다.18일 서울외환시장에서 원·달러 환율은 ...


In [9]:
#save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020_prep_4.csv"
#df.to_csv(save_path, index=False, encoding="utf-8-sig")

# Summarization

- https://huggingface.co/gogamza/kobart-summarization
- KoBart 요약기 사용

In [9]:
# KoBART 요약기 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

# GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 요약 함수 
def summarize_kobart(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    # (KoBART 입력 길이 제한: 1024 tokens)
    text = text[:1024]
    input_ids = tokenizer.encode(text, return_tensors="pt", truncation=True).to(device)
    summary_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약
tqdm.pandas()
df["summary"] = df["content"].progress_apply(summarize_kobart)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
100%|████████████████████████████████████████████████████████████████████████████████| 383/383 [01:12<00:00,  5.31it/s]


In [10]:
df["summary"]

2       한국은행이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...
6                              올해 원/달러 환율은 롤러코스터를 타는 듯했다.
8       30일 오전 원/달러 환율이 하락한 채로 출발한 원/달러 환율은 오전 9시 5분 현...
16      서울 외환시장에서 원/달러 환율은 전 거래일보다 1.7원 오른 달러당 1,158.1...
30      미국과 이란 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/...
                              ...                        
6632    4일 원/달러 환율이 하락 출발한 원/달러 환율은 오전 9시 6분 현재 전날 종가보...
6634    지난 10월 경상수지 흑자가 116억6,000만달러를 기록하며 두 달 연속 100억...
6716                        9일 원/달러 환율이 하루 만에 반락한 채 마감했다.
6896    18일 서울외환시장에서 원·달러 환율은 전날보다 6.4원 상승한 1099.7원에 거...
6978    정부가 22일 중소기업에 집중해 환변동 위험관리 지원책을 내놓은 것은 기업 규모가 ...
Name: summary, Length: 383, dtype: object

In [12]:
df

,date,title,url,content,summary
2,20200101,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치",https://n.news.naver.com/mnews/article/001/001...,외환당국이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...,한국은행이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...
6,20200101,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율은 롤러코스터를 타는 듯했다. 연중 고점과 저점 차이가 110원 ...,올해 원/달러 환율은 롤러코스터를 타는 듯했다.
8,20200101,"위험선호 분위기 속 원/달러 환율 1,150원대로 하락 출발",https://n.news.naver.com/mnews/article/001/001...,30일 오전 원/달러 환율이 하락한 채로 출발했다.이날 서울 외환시장에서 원/달러 ...,30일 오전 원/달러 환율이 하락한 채로 출발한 원/달러 환율은 오전 9시 5분 현...
16,20200102,새해 첫 거래일 원/달러 환율 상승 마감,https://n.news.naver.com/mnews/article/001/001...,새해 첫 거래일인 2일 원/달러 환율이 올랐다.이날 서울 외환시장에서 원/달러 환율...,"서울 외환시장에서 원/달러 환율은 전 거래일보다 1.7원 오른 달러당 1,158.1..."
30,20200103,중동 긴장 고조로 금융시장 불안…주가·환율·금리 '출렁',https://n.news.naver.com/mnews/article/001/001...,미국과 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/달러 ...,미국과 이란 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/...
...,...,...,...,...,...
6632,20201204,미국 부양책 기대에 약달러 심화…원/달러 환율 하락 출발,https://n.news.naver.com/mnews/article/001/001...,4일 원/달러 환율이 하락 출발했다.이날 서울 외환시장에서 원/달러 환율은 오전 9...,4일 원/달러 환율이 하락 출발한 원/달러 환율은 오전 9시 6분 현재 전날 종가보...
6634,20201204,10월 경상 흑자 117억弗 달해 원화 강세 지속되나,https://n.news.naver.com/mnews/article/011/000...,경상흑자 올 해 전망치 540억弗 이미 넘겨 호조세 지난 10월 경상수지 흑자가 1...,"지난 10월 경상수지 흑자가 116억6,000만달러를 기록하며 두 달 연속 100억..."
6716,20201209,"위안화 강세에 원/달러 환율 하루 만에 반락…1,084.8원 마감",https://n.news.naver.com/mnews/article/001/001...,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.이날 서울 외환시장에서 원/달러...,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.
6896,20201218,"원·달러 환율, 장중 1100원 터치…코로나19 악화에 급등 마감",https://n.news.naver.com/mnews/article/015/000...,게티이미지뱅크원·달러 환율이 급등 마감했다.18일 서울외환시장에서 원·달러 환율은 ...,18일 서울외환시장에서 원·달러 환율은 전날보다 6.4원 상승한 1099.7원에 거...


In [13]:
# 추가 필터링
df = df[df['summary'].str.contains('원화|원/달러', na=False)]
print(f"{len(df)}개의 뉴스가 남았습니다.")

268개의 뉴스가 남았습니다.


In [14]:
save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020_prep_s10.csv"
df.to_csv(save_path, index=False, encoding="utf-8-sig")

# Summarization + Prep

(1) 본문을 한 문장으로 요약한 데이터, 
(2) 제목 데이터, 
(3) 본문을 한 문장으로 요약한 후 전처리한 데이터, 
(4) 제목과 본문을 한 문장으로 요약한 데이터

In [11]:
# def clean_summary(text):
#     if not isinstance(text, str):
#         return ""

#     #  (같은 단어 반복) 제거
#     text = re.sub(r'(\b[\w가-힣]{2,10}\s*(원장|기관장)[,]?\s*)\1+', r'\1', text)
#     text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

#     # 중복 쉼표 제거
#     text = re.sub(r',\s*,+', ',', text)

#     # 공백, 점, 따옴표 정리
#     text = re.sub(r'\s+', ' ', text)             
#     text = re.sub(r'\.{2,}', '.', text)            
#     text = re.sub(r'["“”\'‘’]', '', text)          
#     text = text.strip()

#     return text

In [12]:
# df["summary_prep"] = df["summary"].apply(clean_summary)

In [13]:
# df

,date,title,url,content,summary,summary_prep
2,20200101,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치",https://n.news.naver.com/mnews/article/001/001...,외환당국이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...,한국은행이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...,한국은행이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...
6,20200101,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율은 롤러코스터를 타는 듯했다. 연중 고점과 저점 차이가 110원 ...,올해 원/달러 환율은 롤러코스터를 타는 듯했다.,올해 원/달러 환율은 롤러코스터를 타는 듯했다.
8,20200101,"위험선호 분위기 속 원/달러 환율 1,150원대로 하락 출발",https://n.news.naver.com/mnews/article/001/001...,30일 오전 원/달러 환율이 하락한 채로 출발했다.이날 서울 외환시장에서 원/달러 ...,30일 오전 원/달러 환율이 하락한 채로 출발한 원/달러 환율은 오전 9시 5분 현...,30일 오전 원/달러 환율이 하락한 채로 출발한 원/달러 환율은 오전 9시 5분 현...
11,20200101,美中무역분쟁·브렉시트 여전히 대외리스크…美대선도 관건,https://n.news.naver.com/mnews/article/001/001...,연합뉴스 내년 경제전망 설문에 참여한 경제기관장들. 왼쪽부터 최정표 한국개발연구원 ...,"내년 경제전망 설문에 참여한 경제기관장들. 최정표 한국개발연구원 원장, 이재영 대외...","내년 경제전망 설문에 참여한 경제기관장들. 최정표 한국개발연구원 원장, 이재영 대외..."
13,20200101,주식·외환 등 금융시장 30일 폐장…내달 2일 개장,https://n.news.naver.com/mnews/article/001/001...,주식과 외환 등 올해 금융시장이 오는 30일 거래를 마지막으로 폐장한다.27일 한국...,올해 금융시장이 오는 30일 거래를 마지막으로 폐장한다.27일 한국거래소에 따르면 ...,올해 금융시장이 오는 30일 거래를 마지막으로 폐장한다.27일 한국거래소에 따르면 ...
...,...,...,...,...,...,...
6716,20201209,"위안화 강세에 원/달러 환율 하루 만에 반락…1,084.8원 마감",https://n.news.naver.com/mnews/article/001/001...,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.이날 서울 외환시장에서 원/달러...,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.,9일 원/달러 환율이 하루 만에 반락한 채 마감했다.
6756,20201211,"원·달러 환율, 6거래일만에 1090원대로 상승",https://n.news.naver.com/mnews/article/015/000...,게티이미지뱅크원·달러 환율이 연일 상승했다.11일 서울외환시장에서 원·달러 환율은 ...,서울외환시장에서 원·달러 환율은 전날보다 2.6원 상승한 1090.3원에 거래를 마쳤다.,서울외환시장에서 원·달러 환율은 전날보다 2.6원 상승한 1090.3원에 거래를 마쳤다.
6896,20201218,"원·달러 환율, 장중 1100원 터치…코로나19 악화에 급등 마감",https://n.news.naver.com/mnews/article/015/000...,게티이미지뱅크원·달러 환율이 급등 마감했다.18일 서울외환시장에서 원·달러 환율은 ...,18일 서울외환시장에서 원·달러 환율은 전날보다 6.4원 상승한 1099.7원에 거...,18일 서울외환시장에서 원·달러 환율은 전날보다 6.4원 상승한 1099.7원에 거...
6978,20201222,원화 10% 절상때 수송장비 영업이익률 4%p↓…중기업종 취약,https://n.news.naver.com/mnews/article/001/001...,정부가 22일 중소기업에 집중해 환변동 위험관리 지원책을 내놓은 것은 기업 규모가 ...,정부가 22일 중소기업에 집중해 환변동 위험관리 지원책을 내놓은 것은 기업 규모가 ...,정부가 22일 중소기업에 집중해 환변동 위험관리 지원책을 내놓은 것은 기업 규모가 ...
